In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

import azure.storage.blob
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

import sys
sys.path.append('../PIVOT/') # point to wherever the ml-workflow directory is

import utils.data_utils as du
import utils.insert_data as idu
import utils.sql_utils as sq

from tqdm.auto import trange, tqdm
import concurrent.futures

# Need to comment out gray_image = cv2.cvtColor(ri, cv2.COLOR_BGR2GRAY) in du.prepocess_input
# gray_image = ri
from importlib import reload
reload(du)
reload(idu)


<module 'utils.insert_data' from 'C:\\Users\\ysman\\Desktop\\Capstone\\PIVOT\\notebooks\\../PIVOT\\utils\\insert_data.py'>

In [56]:
sq.run_sql_query("select * from Images;")

C:\Users\ysman\AppData\Local\Temp\ipykernel_17204\2612457103.py:1: UserWarning: The query returned empty.
  sq.run_sql_query("select * from Images;")


# Load in the training data for cnn-v1-b3 model

In [96]:
train_data = pd.read_csv("../PIVOT/data/model-summary-cnn-v1-b3.csv")
train_data.head()

,full_path,high_group,pred_label,true_label,is_correct,0,1,2,3,4,5,6,7,8,9,top_5
0,/home/azureuser/data/NAAMES_ml/D20160513T22082...,Other,7,7,1,0.000018,1.146114e-18,2.892286e-07,0.000010,4.914940e-13,0.003955,1.792351e-06,0.995379,0.000604,0.000033,1
1,/home/azureuser/data/NAAMES_ml/D20180402T13445...,Other,0,7,0,0.539550,1.456711e-04,4.600495e-09,0.000224,1.425122e-01,0.006822,3.214117e-11,0.083037,0.226862,0.000847,1
2,/home/azureuser/data/NAAMES_ml/D20160528T22330...,Other,7,7,1,0.000120,5.510693e-15,2.841731e-05,0.000075,6.837943e-09,0.000447,6.920837e-07,0.994058,0.000086,0.005186,1
3,/home/azureuser/data/NAAMES_ml/D20160531T08382...,Other,7,7,1,0.000241,2.925888e-11,4.873408e-06,0.003934,2.065363e-09,0.273014,1.542221e-04,0.718308,0.000597,0.003748,1
4,/home/azureuser/data/NAAMES_ml/D20160526T21432...,Diatom,3,3,1,0.000313,4.062050e-14,6.139785e-05,0.710152,2.181512e-10,0.000211,4.933558e-06,0.287664,0.000222,0.001372,1


In [105]:
np.all(train_data[list(np.arange(10).astype(int).astype(str))].idxmax(axis=1).astype(int) == train_data.pred_label)

True

### Get the order of the class labels and replace the `np.nan` with `Unidentifiable`.

In [50]:
class_labels = train_data.drop_duplicates(subset = ['true_label', 'high_group']).sort_values(by="true_label")['high_group'].values
class_labels[-1] = "Unidentifiable"
class_labels = list(class_labels)

class_labels

['Chloro',
 'Cilliate',
 'Crypto',
 'Diatom',
 'Dictyo',
 'Dinoflagellate',
 'Eugleno',
 'Other',
 'Prymnesio',
 'Unidentifiable']

### Extract the filepath for blob storage and rename the 0-9 index probabilities with their respective class_labels.

In [106]:
train_data["filepath"] = train_data['full_path'].str.split("data/", expand=True)[1]

In [52]:
train_data.head()

,full_path,high_group,pred_label,true_label,is_correct,0,1,2,3,4,5,6,7,8,9,top_5,filepath
0,/home/azureuser/data/NAAMES_ml/D20160513T22082...,Other,7,7,1,0.000018,1.146114e-18,2.892286e-07,0.000010,4.914940e-13,0.003955,1.792351e-06,0.995379,0.000604,0.000033,1,NAAMES_ml/D20160513T220825_IFCB107/IFCB107D201...
1,/home/azureuser/data/NAAMES_ml/D20180402T13445...,Other,0,7,0,0.539550,1.456711e-04,4.600495e-09,0.000224,1.425122e-01,0.006822,3.214117e-11,0.083037,0.226862,0.000847,1,NAAMES_ml/D20180402T134458_IFCB107/IFCB107D201...
2,/home/azureuser/data/NAAMES_ml/D20160528T22330...,Other,7,7,1,0.000120,5.510693e-15,2.841731e-05,0.000075,6.837943e-09,0.000447,6.920837e-07,0.994058,0.000086,0.005186,1,NAAMES_ml/D20160528T223306_IFCB107/IFCB107D201...
3,/home/azureuser/data/NAAMES_ml/D20160531T08382...,Other,7,7,1,0.000241,2.925888e-11,4.873408e-06,0.003934,2.065363e-09,0.273014,1.542221e-04,0.718308,0.000597,0.003748,1,NAAMES_ml/D20160531T083824_IFCB107/IFCB107D201...
4,/home/azureuser/data/NAAMES_ml/D20160526T21432...,Diatom,3,3,1,0.000313,4.062050e-14,6.139785e-05,0.710152,2.181512e-10,0.000211,4.933558e-06,0.287664,0.000222,0.001372,1,NAAMES_ml/D20160526T214329_IFCB107/IFCB107D201...


In [107]:
use_cols = ["filepath", 'pred_label'] + list(np.arange(10).astype(int).astype(str))
train_df = train_data[use_cols]
train_df = train_df.rename(columns = dict(zip(list(np.arange(10).astype(int).astype(str)),class_labels)))
del train_data
train_df.head()

,filepath,pred_label,Chloro,Cilliate,Crypto,Diatom,Dictyo,Dinoflagellate,Eugleno,Other,Prymnesio,Unidentifiable
0,NAAMES_ml/D20160513T220825_IFCB107/IFCB107D201...,7,0.000018,1.146114e-18,2.892286e-07,0.000010,4.914940e-13,0.003955,1.792351e-06,0.995379,0.000604,0.000033
1,NAAMES_ml/D20180402T134458_IFCB107/IFCB107D201...,0,0.539550,1.456711e-04,4.600495e-09,0.000224,1.425122e-01,0.006822,3.214117e-11,0.083037,0.226862,0.000847
2,NAAMES_ml/D20160528T223306_IFCB107/IFCB107D201...,7,0.000120,5.510693e-15,2.841731e-05,0.000075,6.837943e-09,0.000447,6.920837e-07,0.994058,0.000086,0.005186
3,NAAMES_ml/D20160531T083824_IFCB107/IFCB107D201...,7,0.000241,2.925888e-11,4.873408e-06,0.003934,2.065363e-09,0.273014,1.542221e-04,0.718308,0.000597,0.003748
4,NAAMES_ml/D20160526T214329_IFCB107/IFCB107D201...,3,0.000313,4.062050e-14,6.139785e-05,0.710152,2.181512e-10,0.000211,4.933558e-06,0.287664,0.000222,0.001372


In [54]:
train_df_fp = train_df[['filepath']].to_dict(orient='records')

In [110]:
# convert pred_label to string
pred_labels = train_df.pred_label.apply(lambda x: class_labels[x])
train_df['pred_label'] = pred_labels

In [112]:
train_df.head()

,filepath,pred_label,Chloro,Cilliate,Crypto,Diatom,Dictyo,Dinoflagellate,Eugleno,Other,Prymnesio,Unidentifiable
0,NAAMES_ml/D20160513T220825_IFCB107/IFCB107D201...,Other,0.000018,1.146114e-18,2.892286e-07,0.000010,4.914940e-13,0.003955,1.792351e-06,0.995379,0.000604,0.000033
1,NAAMES_ml/D20180402T134458_IFCB107/IFCB107D201...,Chloro,0.539550,1.456711e-04,4.600495e-09,0.000224,1.425122e-01,0.006822,3.214117e-11,0.083037,0.226862,0.000847
2,NAAMES_ml/D20160528T223306_IFCB107/IFCB107D201...,Other,0.000120,5.510693e-15,2.841731e-05,0.000075,6.837943e-09,0.000447,6.920837e-07,0.994058,0.000086,0.005186
3,NAAMES_ml/D20160531T083824_IFCB107/IFCB107D201...,Other,0.000241,2.925888e-11,4.873408e-06,0.003934,2.065363e-09,0.273014,1.542221e-04,0.718308,0.000597,0.003748
4,NAAMES_ml/D20160526T214329_IFCB107/IFCB107D201...,Diatom,0.000313,4.062050e-14,6.139785e-05,0.710152,2.181512e-10,0.000211,4.933558e-06,0.287664,0.000222,0.001372


### Insert train data filepaths to SQL.

In [ ]:
idu.initial_ingestion(image_filepaths=train_df.filepath.values)

In [67]:
images_that_no_exist = [
    "NAAMES_ml/D20160513T121618_IFCB107/IFCB107D20160513T121618P01929.png",
    "NAAMES_ml/D20160513T121618_IFCB107/IFCB107D20160513T121618P02763.png",
    "NAAMES_ml/D20160513T121618_IFCB107/IFCB107D20160513T121618P00606.png",
    "NAAMES_ml/D20160513T121618_IFCB107/IFCB107D20160513T121618P03858.png",
    "NAAMES_ml/D20160513T121618_IFCB107/IFCB107D20160513T121618P01726.png",
]

In [76]:
image_ids = sq.run_sql_query("select * from images;")

remaining_images = list(set(train_df.filepath.values) - set(image_ids.filepath.values) - set(images_that_no_exist))
len(remaining_images)

0

Insert images that failed the first time around

In [ ]:
idu.initial_ingestion(image_filepaths=remaining_images)

### Gather I_IDs for each of the imagery we inserted. Then, join the with the class probs and insert the predictions into the Predictions table

In [82]:
image_ids.shape

(399775, 2)

In [83]:
train_df.shape

(399780, 11)

In [152]:
train_df_merged = train_df.merge(image_ids, on="filepath", how="inner").sort_values(by="i_id")

In [237]:
print(train_df_merged.shape)
train_df_merged.head()

(399775, 13)


,filepath,pred_label,Chloro,Cilliate,Crypto,Diatom,Dictyo,Dinoflagellate,Eugleno,Other,Prymnesio,Unidentifiable,i_id
19000,NAAMES_ml/D20151107T140949_IFCB107/IFCB107D201...,Unidentifiable,0.00000120,0.00000001,0.00000002,0.00003709,0.00000141,0.00002146,0.00000000,0.01926701,0.00008079,0.98059100,1
19001,NAAMES_ml/D20160519T044744_IFCB107/IFCB107D201...,Chloro,0.53398550,0.00000000,0.03066084,0.00009367,0.00000002,0.00664890,0.00011772,0.42713898,0.00028626,0.00106817,2
19002,NAAMES_ml/D20160528T220927_IFCB107/IFCB107D201...,Other,0.00003250,0.00000000,0.00000030,0.00012561,0.00000000,0.05113956,0.00001469,0.93242340,0.01558691,0.00067707,3
19003,NAAMES_ml/D20160517T095745_IFCB107/IFCB107D201...,Other,0.00146442,0.00000003,0.00064200,0.00134276,0.00000061,0.07592753,0.00017256,0.73473006,0.18398885,0.00173112,4
19004,NAAMES_ml/D20160513T005054_IFCB107/IFCB107D201...,Other,0.00000250,0.00000000,0.00000001,0.00000465,0.00000000,0.00342124,0.00000075,0.99601930,0.00054119,0.00001038,5


In [223]:
# combine the class_probs into 1 dictionary per image
prob_dicts = (train_df_merged[class_labels]
              .rename(columns = dict(zip(class_labels, list(np.arange(10).astype(int)))))
              .to_dict(orient = 'records')
             )

In [236]:
# Gather the necessary variables in the right order for inserting into the Predictions Table
pred_table_train = train_df_merged[['i_id', 'pred_label']]
pred_table_train['class_prob'] = np.array(prob_dicts, dtype=str)
# assign model id as 1
pred_table_train['m_id'] = 1
# reorder for insertion
pred_table_train = pred_table_train[['m_id', 'i_id', 'class_prob', 'pred_label']]

pred_table_train.head()

,m_id,i_id,class_prob,pred_label
19000,1,1,"{0: 1.2007497e-06, 1: 9.2799395e-09, 2: 1.8873...",Unidentifiable
19001,1,2,"{0: 0.5339855, 1: 1.2086762e-10, 2: 0.03066084...",Chloro
19002,1,3,"{0: 3.2504617e-05, 1: 1.1344883e-11, 2: 2.9553...",Other
19003,1,4,"{0: 0.0014644222, 1: 2.9109668e-08, 2: 0.00064...",Other
19004,1,5,"{0: 2.5031059e-06, 1: 6.138133999999999e-20, 2...",Other


#### Insert model into models table first.

In [242]:
cnn_class_map = dict(zip(list(np.arange(10).astype(int)),class_labels))

du.insert_data(table_name='models', data = {"model_name":"model-cnn-v1-b3",
                                            "model_link": "https://basemodel-endpoint.westus2.inference.ml.azure.com/score",
                                            "class_map": str(cnn_class_map)
})

Decimal('1')

In [243]:
sq.run_sql_query("select * from models;")

,m_id,model_name,model_link,class_map
0,1,model-cnn-v1-b3,https://basemodel-endpoint.westus2.inference.m...,"{0: 'Chloro', 1: 'Cilliate', 2: 'Crypto', 3: '..."


You can read in the str(dictionary) to python with ast.literal_eval(string)
```python
    import ast
    ast.literal_eval(string)
```

#### Insert the raw data into predictions

In [252]:
a = sq.run_sql_query("select top(10) * from predictions;")

In [244]:
pred_table_train_lst = pred_table_train.to_dict(orient='records')

In [249]:
idu.bulk_insert_data(table_name="predictions", data=pred_table_train_lst)

0it [00:00, ?it/s]

In [238]:
sq.run_sql_query("""
SELECT column_name, data_type, character_maximum_length
FROM information_schema.columns
WHERE table_name = 'models';
""")

,column_name,data_type,character_maximum_length
0,m_id,int,NaN
1,model_name,varchar,255.00000000
2,model_link,varchar,-1.00000000
3,class_map,varchar,-1.00000000


# Checkout Non-Train Data

In [177]:
# load all new data
new_data = pd.read_parquet("../PIVOT/data/inventory_df_with_probs.parquet.gzip")

print(new_data.shape)
new_data.head()

(1056000, 15)


,Index,image_path,pred_label,pred_class,index,0,1,2,3,4,5,6,7,8,9
0,0,/Users/alisonchase/Documents/IFCB/NAAMES/NAAME...,7,Other,0,0.00318988,0.00000000,0.00008589,0.00017344,0.00000009,0.00327110,0.00000809,0.95753310,0.03550129,0.00023711
1,1,/Users/alisonchase/Documents/IFCB/NAAMES/NAAME...,7,Other,1,0.00755106,0.00000051,0.00035928,0.00260129,0.00000253,0.20422825,0.00101794,0.71885103,0.06120521,0.00418288
2,2,/Users/alisonchase/Documents/IFCB/NAAMES/NAAME...,9,None,2,0.00013524,0.00000000,0.00001258,0.00022039,0.00002652,0.00010345,0.00000018,0.39091730,0.00037549,0.60820890
3,3,/Users/alisonchase/Documents/IFCB/NAAMES/NAAME...,7,Other,3,0.00112294,0.00000000,0.00004770,0.00016701,0.00000000,0.01313227,0.00001304,0.98447030,0.00025867,0.00078803
4,4,/Users/alisonchase/Documents/IFCB/NAAMES/NAAME...,7,Other,4,0.00010031,0.00000001,0.00000009,0.00810581,0.00000001,0.34370705,0.00002030,0.64535946,0.00038832,0.00231878


In [201]:
# extract the bloc storage path
new_data['filepath'] = new_data.image_path.str.split("NAAMES/", expand=True)[1]
# remove data that was already in train
new_df = new_data.drop(columns = ['image_path']).merge(image_ids, on ='filepath', how ='outer', indicator=True).query('_merge=="left_only"')
# gather only necessary columns
new_df = new_df.drop(columns = ['Index', 'pred_class', 'index', 'i_id', '_merge'])
# convert pred_label into int
new_df['pred_label'] = new_df['pred_label'].astype(int)
# convert pred_label to string that will be written to sql
pred_labels = new_df.pred_label.apply(lambda x: class_labels[x])
new_df['pred_label'] = pred_labels

In [259]:
new_df.head()

,pred_label,0,1,2,3,4,5,6,7,8,9,filepath
0,Other,0.00318988,0.00000000,0.00008589,0.00017344,0.00000009,0.00327110,0.00000809,0.95753310,0.03550129,0.00023711,NAAMES_ml/D20160524T225721_IFCB107/IFCB107D201...
1,Other,0.00755106,0.00000051,0.00035928,0.00260129,0.00000253,0.20422825,0.00101794,0.71885103,0.06120521,0.00418288,NAAMES_ml/D20160524T225721_IFCB107/IFCB107D201...
2,Unidentifiable,0.00013524,0.00000000,0.00001258,0.00022039,0.00002652,0.00010345,0.00000018,0.39091730,0.00037549,0.60820890,NAAMES_ml/D20160524T225721_IFCB107/IFCB107D201...
3,Other,0.00112294,0.00000000,0.00004770,0.00016701,0.00000000,0.01313227,0.00001304,0.98447030,0.00025867,0.00078803,NAAMES_ml/D20160524T225721_IFCB107/IFCB107D201...
4,Other,0.00010031,0.00000001,0.00000009,0.00810581,0.00000001,0.34370705,0.00002030,0.64535946,0.00038832,0.00231878,NAAMES_ml/D20160524T225721_IFCB107/IFCB107D201...


#### Insert table to images

In [210]:
new_imagepaths = new_df[['filepath']].to_dict(orient="records")

In [214]:
%%time
idu.bulk_insert_data(table_name="images", data=new_imagepaths)

0it [00:00, ?it/s]

CPU times: total: 27.7 s
Wall time: 6min 37s


In [216]:
image_ids = sq.run_sql_query("select * from images;")

In [217]:
image_ids

,i_id,filepath
0,1,NAAMES_ml/D20151107T140949_IFCB107/IFCB107D201...
1,2,NAAMES_ml/D20160519T044744_IFCB107/IFCB107D201...
2,3,NAAMES_ml/D20160528T220927_IFCB107/IFCB107D201...
3,4,NAAMES_ml/D20160517T095745_IFCB107/IFCB107D201...
4,5,NAAMES_ml/D20160513T005054_IFCB107/IFCB107D201...
...,...,...
1382662,1382664,NAAMES_ml/D20151105T224631_IFCB107/IFCB107D201...
1382663,1382665,NAAMES_ml/D20151105T224631_IFCB107/IFCB107D201...
1382664,1382666,NAAMES_ml/D20151105T224631_IFCB107/IFCB107D201...
1382665,1382667,NAAMES_ml/D20151105T224631_IFCB107/IFCB107D201...


### Gather image_ids and prepare table for inserting into predictions

In [268]:
pred_table_new = new_df.merge(image_ids, on='filepath', how ='inner').sort_values(by="i_id")

In [269]:
pred_table_new.head()

,pred_label,0,1,2,3,4,5,6,7,8,9,filepath,i_id
0,Other,0.00318988,0.00000000,0.00008589,0.00017344,0.00000009,0.00327110,0.00000809,0.95753310,0.03550129,0.00023711,NAAMES_ml/D20160524T225721_IFCB107/IFCB107D201...,399777
1,Other,0.00755106,0.00000051,0.00035928,0.00260129,0.00000253,0.20422825,0.00101794,0.71885103,0.06120521,0.00418288,NAAMES_ml/D20160524T225721_IFCB107/IFCB107D201...,399778
2,Unidentifiable,0.00013524,0.00000000,0.00001258,0.00022039,0.00002652,0.00010345,0.00000018,0.39091730,0.00037549,0.60820890,NAAMES_ml/D20160524T225721_IFCB107/IFCB107D201...,399779
3,Other,0.00112294,0.00000000,0.00004770,0.00016701,0.00000000,0.01313227,0.00001304,0.98447030,0.00025867,0.00078803,NAAMES_ml/D20160524T225721_IFCB107/IFCB107D201...,399780
4,Other,0.00010031,0.00000001,0.00000009,0.00810581,0.00000001,0.34370705,0.00002030,0.64535946,0.00038832,0.00231878,NAAMES_ml/D20160524T225721_IFCB107/IFCB107D201...,399781


In [270]:
# combine the class_probs into 1 dictionary per image
prob_dicts_new = (pred_table_new[list(np.arange(10).astype(int).astype(str))]
              .rename(columns = dict(zip(list(np.arange(10).astype(int).astype(str)), list(np.arange(10).astype(int)))))
              .to_dict(orient = 'records')
             )

In [272]:
# Gather the necessary variables in the right order for inserting into the Predictions Table
pred_table_new = pred_table_new[['i_id', 'pred_label']]
pred_table_new['class_prob'] = np.array(prob_dicts_new, dtype=str)
# assign model id as 1
pred_table_new['m_id'] = 1
# reorder for insertion
pred_table_new = pred_table_new[['m_id', 'i_id', 'class_prob', 'pred_label']]

pred_table_new.head()

C:\Users\ysman\AppData\Local\Temp\ipykernel_17204\2025785591.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_table_new['class_prob'] = np.array(prob_dicts_new, dtype=str)
C:\Users\ysman\AppData\Local\Temp\ipykernel_17204\2025785591.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_table_new['m_id'] = 1


,m_id,i_id,class_prob,pred_label
0,1,399777,"{0: 0.0031898795, 1: 2.0788812e-11, 2: 8.58863...",Other
1,1,399778,"{0: 0.007551055, 1: 5.1269853e-07, 2: 0.000359...",Other
2,1,399779,"{0: 0.00013524287, 1: 1.2006405e-11, 2: 1.2578...",Unidentifiable
3,1,399780,"{0: 0.0011229421, 1: 1.05487035e-14, 2: 4.7695...",Other
4,1,399781,"{0: 0.00010030835, 1: 7.8161495e-09, 2: 8.5463...",Other


In [273]:
pred_table_new_lst = pred_table_new.to_dict(orient='records')

In [274]:
len(pred_table_new_lst)

982892

In [275]:
idu.bulk_insert_data(table_name="predictions", data=pred_table_new_lst)

0it [00:00, ?it/s]

# Put Random Subset for test evaluation

Insert into dissimilarity and model table first to prevent FK errors.

In [ ]:
# insert 0 representing random test_data
sq.run_sql_query(
"""
SET IDENTITY_INSERT dissimilarity ON;

INSERT INTO dissimilarity (d_id, name, formula)
VALUES (0, 'random_sample', 'none');
SET IDENTITY_INSERT dissimilarity OFF;
"""
)

In [312]:
# insert 0 representing test_data
sq.run_sql_query(
"""
SET IDENTITY_INSERT models ON;


INSERT INTO models (m_id, model_name, model_link, class_map)
VALUES (0, 'random_sample', 'none', 'none');
SET IDENTITY_INSERT models OFF;
"""
)

C:\Users\ysman\AppData\Local\Temp\ipykernel_17204\580666205.py:2: UserWarning: The query returned empty.
  sq.run_sql_query(


Run stored procedure to gather test set of 100,000 images.

In [372]:
sq.create_alter_stored_procedure('GENERATE_RANDOM_TEST_SET')

Using preset file to create procedure GENERATE_RANDOM_TEST_SET: C:\Users\ysman\Desktop\Capstone\PIVOT\PIVOT\utils\stored_procedures\Generate_Random_Test_Set.sql


Get list of train_ids that can't be used for metric evaluation.

In [305]:
train_ids = [i for i in range(1, 399777)]

In [373]:
sq.generate_random_evaluation_set(train_ids = train_ids)

Using argument workaround with execute because IMAGE_IDS has length 2687326.


In [377]:
sq.run_sql_query("select count(*) from metrics;")

,
0,100000


### Prepare for insertion into the Metrics table

Insert methods into the dissimilarity table.

In [378]:
sq.run_sql_query(
"""
INSERT INTO dissimilarity (name, formula)
VALUES ('entropy', '-x.T @ np.nan_to_num(np.log(x))')
"""
)

C:\Users\ysman\AppData\Local\Temp\ipykernel_17204\2236948168.py:1: UserWarning: The query returned empty.
  sq.run_sql_query(


In [379]:
sq.run_sql_query("select * from dissimilarity")

,d_id,name,formula
0,0,random_sample,none
1,1,entropy,-x.T @ np.nan_to_num(np.log(x))


### train_data

In [386]:
def least_confident_score(x):
    return 1 - np.max(x)

def least_margin_score(x):
    sort_x = np.sort(x)
    return 1 - (sort_x[-1] - sort_x[-2])

def entropy_score(x):
    return -x.T @ np.nan_to_num(np.log(x))
def get_score(x, f):
    try:
        s = f(x)
    except:
        print(f"Error computing {f.__name__}")
        s = -1
    return s

In [380]:
train_df_merged.head()

,filepath,pred_label,Chloro,Cilliate,Crypto,Diatom,Dictyo,Dinoflagellate,Eugleno,Other,Prymnesio,Unidentifiable,i_id
19000,NAAMES_ml/D20151107T140949_IFCB107/IFCB107D201...,Unidentifiable,0.00000120,0.00000001,0.00000002,0.00003709,0.00000141,0.00002146,0.00000000,0.01926701,0.00008079,0.98059100,1
19001,NAAMES_ml/D20160519T044744_IFCB107/IFCB107D201...,Chloro,0.53398550,0.00000000,0.03066084,0.00009367,0.00000002,0.00664890,0.00011772,0.42713898,0.00028626,0.00106817,2
19002,NAAMES_ml/D20160528T220927_IFCB107/IFCB107D201...,Other,0.00003250,0.00000000,0.00000030,0.00012561,0.00000000,0.05113956,0.00001469,0.93242340,0.01558691,0.00067707,3
19003,NAAMES_ml/D20160517T095745_IFCB107/IFCB107D201...,Other,0.00146442,0.00000003,0.00064200,0.00134276,0.00000061,0.07592753,0.00017256,0.73473006,0.18398885,0.00173112,4
19004,NAAMES_ml/D20160513T005054_IFCB107/IFCB107D201...,Other,0.00000250,0.00000000,0.00000001,0.00000465,0.00000000,0.00342124,0.00000075,0.99601930,0.00054119,0.00001038,5


In [388]:
%%time
entropy_train = train_df_merged[['i_id']]
entropy_train['m_id'] = 1
entropy_train['d_id'] = 1
entropy_train['d_value'] = train_df_merged[class_labels].apply(lambda x: get_score(x,entropy_score), axis=1)


<timed exec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\ysman\anaconda3\envs\pivot-modAL\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


CPU times: total: 59.9 s
Wall time: 1min


<timed exec>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [400]:
%%time
idu.bulk_insert_data(table_name="metrics", data=entropy_train.to_dict(orient='records'))

0it [00:00, ?it/s]

CPU times: total: 16.6 s
Wall time: 3min 27s


## New images

In [4]:
new_scores= pd.read_parquet("../pivot/data/inventory_df_with_scores.parquet.gzip")
new_scores.shape

(1056000, 8)

In [6]:
new_scores.tail()

,Index,image_path,pred_label,pred_class,index,entropy_score,least_confident_score,least_margin_score
1055995,1103035,/Users/alisonchase/Documents/IFCB/NAAMES/NAAME...,2,Crypto,1103035,0.234404,0.057854,0.112909
1055996,1103036,/Users/alisonchase/Documents/IFCB/NAAMES/NAAME...,7,Other,1103036,1.129720,0.399378,0.606505
1055997,1103037,/Users/alisonchase/Documents/IFCB/NAAMES/NAAME...,7,Other,1103037,1.289111,0.385930,0.545256
1055998,1103038,/Users/alisonchase/Documents/IFCB/NAAMES/NAAME...,0,Chloro,1103038,0.526977,0.174938,0.332709
1055999,1103039,/Users/alisonchase/Documents/IFCB/NAAMES/NAAME...,7,Other,1103039,0.795194,0.474722,0.931747


In [12]:
# gather IDs for images
image_ids = sq.run_sql_query("select * from images;")
image_ids.shape

(1382667, 2)

In [24]:
train_subset = new_scores[['filepath', 'entropy_score']].merge(image_ids, on='filepath', how='inner').query("i_id<=399776")

In [23]:
# join image IDs to table and remove train data that might overlap
new_scores['filepath'] = new_scores['image_path'].str.split("NAAMES/", expand=True)[1]
entropy_new = new_scores[['filepath', 'entropy_score']].merge(image_ids, on='filepath', how='inner').query("i_id>399776")

# assign model and d_id
entropy_new['m_id'] = 1
entropy_new['d_id'] = 1
# reorder to relevant info
entropy_new = entropy_new.rename(columns={"entropy_score":"d_value"})[['i_id','m_id', 'd_id', 'd_value']]
print(entropy_new.shape)
entropy_new.head()

(982892, 4)


,i_id,m_id,d_id,d_value
0,399777,1,1,0.201498
1,399778,1,1,0.817881
2,399779,1,1,0.676992
3,399780,1,1,0.089775
4,399781,1,1,0.706994


In [27]:
%%time
idu.bulk_insert_data(table_name="metrics", data=entropy_new.to_dict(orient='records'))

0it [00:00, ?it/s]

CPU times: total: 27.8 s
Wall time: 8min 38s


In [32]:
import utils

In [33]:
reload(utils)

<module 'utils' from 'C:\\Users\\ysman\\Desktop\\Capstone\\PIVOT\\notebooks\\../PIVOT\\utils\\__init__.py'>

In [36]:
sq.create_alter_stored_procedure('GENERATE_IMAGES_TO_PREDICT', file_path='../pivot/utils/stored_procedures/Images_To_Predict.sql' )

Using custom file to create procedure GENERATE_IMAGES_TO_PREDICT: ../pivot/utils/stored_procedures/Images_To_Predict.sql


In [37]:
sq.create_alter_stored_procedure('GENERATE_IMAGES_TO_METRIZE', file_path='../pivot/utils/stored_procedures/Images_To_Metrize.sql' )

Using custom file to create procedure GENERATE_IMAGES_TO_METRIZE: ../pivot/utils/stored_procedures/Images_To_Metrize.sql


# Load the Existing Labels for Train Data

In [41]:
train_data = pd.read_csv("../PIVOT/data/model-summary-cnn-v1-b3.csv")
train_data["filepath"] = train_data['full_path'].str.split("data/", expand=True)[1]

In [43]:
train_data = train_data[['filepath', 'true_label']].merge(image_ids, on='filepath', how='inner')

In [44]:
train_data.shape

(399775, 3)

Gather the class label mapping from the `models` table.

In [49]:
class_map = sq.run_sql_query("select * from models where m_id = 1;")

In [65]:
import ast
class_map = ast.literal_eval(class_map.class_map[0])
class_map

{0: 'Chloro',
 1: 'Cilliate',
 2: 'Crypto',
 3: 'Diatom',
 4: 'Dictyo',
 5: 'Dinoflagellate',
 6: 'Eugleno',
 7: 'Other',
 8: 'Prymnesio',
 9: 'Unidentifiable'}

In [63]:
# use mapper to convert ints to varchar labels.
train_data['label'] = train_data['true_label'].map(class_map)

Insert an "unknown labeler" to the users table to allow us to insert the training data labels with their respective experience.
I set it as 3 because I assume it was grad students who did it. 

In [68]:
sq.run_sql_query("""
Insert into users (email, name, experience, lab)
VALUES ('initial@unknown.com', 'Initial Labelers', 3, 'Labelers who annotated data for model cnn-v1-b3');
""")

C:\Users\ysman\AppData\Local\Temp\ipykernel_10404\2769406685.py:1: UserWarning: The query returned empty.
  sq.run_sql_query("""


In [69]:
sq.run_sql_query("select * from users;")

,u_id,email,name,experience,lab
0,1,initial@unknown.com,Initial Labelers,3,Labelers who annotated data for model cnn-v1-b3


Get the schema for `labels`.

In [58]:
sq.run_sql_query("""
SELECT column_name, data_type, character_maximum_length
FROM information_schema.columns
WHERE table_name = 'labels';
""")

,column_name,data_type,character_maximum_length
0,i_id,int,NaN
1,u_id,int,NaN
2,weight,int,NaN
3,date,datetime,NaN
4,label,varchar,255.0


Now, let's assing the u_id for each of these labels to be 1.

In [70]:
from datetime import datetime

In [86]:
train_data['u_id'] = 1
# default user experience is 3
train_data['weight'] = 3
# set labeling date to last year
train_data['date'] = str(datetime(2023, 1, 1, 0, 0, 0))

# reorder columns
train_data = train_data[['i_id', 'u_id', 'weight', 'date','label']]

train_data.head()

,i_id,u_id,weight,date,label
0,16001,1,3,2023-01-01 00:00:00,Other
1,16002,1,3,2023-01-01 00:00:00,Other
2,16003,1,3,2023-01-01 00:00:00,Other
3,16004,1,3,2023-01-01 00:00:00,Other
4,16005,1,3,2023-01-01 00:00:00,Diatom


In [87]:
%%time
# insert into labels
idu.bulk_insert_data(table_name="labels", data=train_data.to_dict(orient='records'))

0it [00:00, ?it/s]

CPU times: total: 54.2 s
Wall time: 4min 23s
